# Исследование надёжности заёмщиков - анализ банковских данных

**Задача:** Заказчик — кредитный отдел банка. Нужно провести анализ данных и выяснить, влияет ли семейное положение и количество детей клиента на погашение кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга (системы оценки способности потенциального заёмщика вернуть кредит банку).

**Входные данные от банка:** Статистика о платёжеспособности клиентов.

<a id='content'></a>
## Оглавление 
[Предварительный анализ данных](#preprocessing)

[Предобработка данных](#preprocessing_part2)
 - [Обработка пропусков](#2-2)
 - [Замена типа данных](#2-3)
 - [Обработка дубликатов](#2-4)
 - [Лемматизация](#2-5) 
 - [Категоризация данных](#2-6) 
  
[Исследовательский анализ данных](#EDA)
 - [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#3-1)
 - [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#3-2)
 - [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#3-3)
 - [Как разные цели кредита влияют на его возврат в срок?](#3-4)
 
[Общий вывод](#outcome)

<a id = 'preprocessing'></a>
## Предварительный анализ данных

In [ ]:
# импорт библиотек
import pandas as pd
from pymystem3 import Mystem

In [1]:
# чтение данных из файла
df = pd.read_csv('/datasets/data.csv')

# информация о данных
df.info()

# вывод данных
display(df)

# значения столбца income_type
display(df.groupby('income_type')['income_type'].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

<b>Вывод:</b> столбец days_employed принимает положительные и отрицательные значения. Причем, положительные значения days_employed соотвествуют income_type 'пенсионер'. Так же, зачения days_employed для пенсионеров завышены и нереальны (343937 дней это 942 года). Стоит обратить на это внимание операторов с целью выявить причину и исправить данную ошибку. Использование текущих значений для расчетов может исказить результат. Однако, для проверки нашей гипотезы данная информация не требуется <br><br>

Столбец education_id кодирует education. Столбец family_status_id кодирует family_status. Стоит вынести эту кодировку в отдельную таблицу<br><br>

В столбце 'education' используются разный регистр букв. Это приведет к появлению неявных дубликатов

[К оглавлению](#content)

<a id = 'preprocessing_part2'></a>
## Предобработка данных

<a id = '2-2'></a>
### Обработка пропусков

In [2]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<b>Вывод:</b> пропущенны значения в столбцах 'days_employed' и 'total_income'. Поскольку характер распределения значений не известен, пропущенные значения будут заменены на медианные. Вероятно, пропуски появились при выгрузке или существует системная ошибка. Стоит уточнить у специалистов, которые подготавливали базу данных

Для расчета медианного значения days_employed_median не будут использоваться значения income_type 'пенсионер', поскольку данные значения нереалистичны

In [3]:
# расчет медианных значений
days_employed_median = df[df['income_type'] != 'пенсионер']['days_employed'].median()
total_income_median = df['total_income'].median()

# функция замены отрицательных значений на положительные в столбце days_employed 
def negative_to_positive(item):
    if item < 0:
        item *= -1
    return item

# замена отрицательных значений на положительные в столбце days_employed
df['days_employed'] = df['days_employed'].apply(negative_to_positive)     

# замена пропущенных значений на медианные
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

[К оглавлению](#content)

<a id = '2-3'></a>
### Замена типа данных

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


<b>Вывод:</b> в столбцах days_employed и total_income значения типа float заменены на int

[К оглавлению](#content)

<a id = '2-4'></a>
### Обработка дубликатов

In [5]:
# обработка дубликатов в столбце education по причине разного регистра 
display(df.groupby('education')['education'].count())
df['education'] = df['education'].str.lower()
display(df.groupby('education')['education'].count())

# проверка столбца gender на дубликаты 
display(df['gender'].value_counts())

# проверка столбца children. Замена значения -1 на 1, 20 на медианное значение
display(df['children'].value_counts())

df['children'] = df['children'].replace(-1, 1)
children_median =  df['children'].median()
df['children'] = df['children'].replace(20, children_median)
display(df['children'].value_counts())

# проверка столбца family на дубликаты
display(df['family_status'].value_counts())

# проверка столбца income_type на дубликаты
display(df['income_type'].value_counts())

# проверка столбца purpose на дубликаты
display(df['purpose'].value_counts())

education
ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [12]:
# проеврка на дубликаты строк
df.duplicated().sum()

71

In [13]:
df.loc[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_id
2849,0,-1629,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи,недвижимость,2
3290,0,-1629,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу,свадьба,2
4182,1,-1629,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба,свадьба,2
4851,0,-1629,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба,свадьба,2
5557,0,-1629,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу,свадьба,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,-1629,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование,образование,2
21032,0,-1629,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием,образование,2
21132,0,-1629,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью,недвижимость,2
21281,1,-1629,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости,недвижимость,2


<b>Вывод: </b> в столбце education значения приведены к одному регистру.

В столбце gender обнаружено значение пола 'XNA'. Скорее всего это пропуск. Данное значение не будет использовано для анализа

В столбце children два атрефакта: -1 и 20. -1 заменила на 1 (скорее всего -1 это опечатка). 20 заменила на медианное значение, т к нет информации, по какой причине возникла эта опечатка (возможно это 2)

Столбцы gender, children, family_status и income_type были проверены на дубликаты при помощи метода value_counts(). Дубликаты обнаружены не были
    
Проверка на дубликаты в столбце education была выполнена при помощи метода count(). Поскольку разница была только в регистре, все значения были приведены к единому регистру. Тем самым дубликаты были удалены

Столбец purpose был проверен на дубликаты при помощи метода value_counts(). Для избавления от дубликатов нужна лемматизация

[К оглавлению](#content)

<a id = '2-5'></a>
### Лемматизация

In [6]:
m = Mystem()

estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образов']

df['purpose_category'] = 0

def purpose_categoryzer(lemmas, category):
    lemmas = '|'.join(lemmas)
    index = df[df['purpose'].str.lower().str.contains(lemmas)].index.to_list()
    for i in index:
        df.loc[i, 'purpose_category'] = category
    return df           
    

purpose_categoryzer(wedding, 'свадьба')
purpose_categoryzer(estate, 'недвижимость')
purpose_categoryzer(auto, 'автомобиль')
purpose_categoryzer(education, 'образование')

display(df['purpose_category'].value_counts())

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_category, dtype: int64

<b>Вывод:</b> были выявлены 4 семантические группы: свадьба, недвижимость и жилье, автомобиль и образование

[К оглавлению](#content)

<a id = '2-6'></a>
### Категоризация данных

In [7]:
display(df.groupby('children')['debt'].count())
display(df.groupby('family_status')['debt'].count())

# категоризация по доходу с шагом 100000
df['total_income_id'] = 0

def income_categoryzer():
    for i in range(len(df['total_income'])):
        if df.loc[i, 'total_income'] <= 100000: 
            df.loc[i, 'total_income_id'] = 1
        elif 100000 < df.loc[i, 'total_income'] <= 200000:
            df.loc[i, 'total_income_id'] = 2
        elif 200000 < df.loc[i, 'total_income'] <= 300000:
            df.loc[i, 'total_income_id'] = 3
        elif 300000 < df.loc[i, 'total_income'] <= 400000:
            df.loc[i, 'total_income_id'] = 4
        elif 400000 < df.loc[i, 'total_income'] <= 500000:
            df.loc[i, 'total_income_id'] = 5
        else:
            df.loc[i, 'total_income_id'] = 6
    return df

income_categoryzer()
df['total_income_id'].value_counts()


children
0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: debt, dtype: int64

family_status
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: debt, dtype: int64

2    11996
1     4463
3     3584
4      954
5      306
6      222
Name: total_income_id, dtype: int64

<b>Вывод:</b> предстоит проанализировать влияние семейного положения и количества детей клиента на факт погашения кредита в срок.Количество детей и семейное положение категоризировать не нужно. Уровень дохода был категоризирован с шагом 100000 рублей

[К оглавлению](#content)

<a id='EDA'></a>
## Исследовательский анализ данных

<a id = '3-1'></a>
### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
debt_children = pd.DataFrame()
debt_children['count_children'] = df.groupby('children')['debt'].count()
debt_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_children['result_children'] = debt_children['sum_children'] / debt_children['count_children'] 
display(debt_children)

# для проверки сравним sum_children и общим числом задержек по кредиту
display(df.groupby('debt')['debt'].count())

,count_children,sum_children,result_children
children,,,
0,14225,1071,0.075290
1,4865,445,0.091470
2,2055,194,0.094404
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


debt
0    19784
1     1741
Name: debt, dtype: int64

<b>Вывод: </b>заемщики без детей реже имеют задолжность по кредитам, чем заемщики с детьми. В целом, можно говорить о восходящем тренде - чем больше детей, тем чаще встречается задолжность по кредитам. Исключение составляют заемщики с тремя детьми и с пятью детьми, что может быть связано с малой выборкой. По заемщикам с пятью детьми недостаточно исходных данных для анализа. 

[К оглавлению](#content)

<a id = '3-2'></a>
### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
debt_family = pd.DataFrame()
debt_family['count_family'] = df.groupby('family_status')['debt'].count()
debt_family['sum_family'] = df.groupby('family_status')['debt'].sum()
debt_family['debt_family'] = debt_family['sum_family'] / debt_family['count_family'] 

debt_family.sort_values('debt_family', ascending = False) 

,count_family,sum_family,debt_family
family_status,,,
Не женат / не замужем,2813,274,0.097405
гражданский брак,4177,388,0.092890
женат / замужем,12380,931,0.075202
в разводе,1195,85,0.071130
вдовец / вдова,960,63,0.065625


<b>Вывод: </b>заемщики не состоящие в официальных отношениях чаще имеют задолжности по кредитам, чем заемщики состоящие или состоявшие в официальных отношениях. Меньше всего задолжность по кредиту встречается в категории вдовец/вдова

[К оглавлению](#content)

<a id = '3-3'></a>
### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
debt_income = pd.DataFrame()

debt_income['count_income'] = df.groupby('total_income_id')['debt'].count()
debt_income['sum_income'] = df.groupby('total_income_id')['debt'].sum()
debt_income['debt_income'] = debt_income['sum_income'] / debt_income['count_income'] 

debt_income.sort_values('debt_income', ascending = False) 

,count_income,sum_income,debt_income
total_income_id,,,
2,11996,1029,0.085779
1,4463,354,0.079319
4,954,75,0.078616
3,3584,252,0.070312
6,222,14,0.063063
5,306,17,0.055556


<b>Вывод: </b>существует нисходящий тренд: чем выше уровень дохода, чем меньше число задолжностей по кредиту.

[К оглавлению](#content)

<a id = '3-4'></a>
### Как разные цели кредита влияют на его возврат в срок?

In [11]:
debt_purpose = pd.DataFrame()

debt_purpose['count_purpose'] = df.groupby('purpose_category')['debt'].count()
debt_purpose['sum_purpose'] = df.groupby('purpose_category')['debt'].sum()
debt_purpose['debt_purpose'] = debt_purpose['sum_purpose'] / debt_purpose['count_purpose'] 

debt_purpose.sort_values('debt_purpose', ascending = False) 

,count_purpose,sum_purpose,debt_purpose
purpose_category,,,
автомобиль,4315,403,0.093395
образование,4022,370,0.091994
свадьба,2348,186,0.079216
недвижимость,10840,782,0.072140


<b>Вывод: </b>минимальное количество задолжностей встречается среди заемщиков, которые брали кредит на недвижимость. Заемщики, которые берут кредит на покупку автомобиля или на образование чаще имеют просрочку по кредиту, чем заемщики, которые берут кредит на недвижимость и свадьбу. 

[К оглавлению](#content)

<a id='outcome'></a>
## Общий вывод

 - Семейный статус связан с вероятностью просрочки по кредиту. Заемщики, состоящие в браке имеют меньше задолжностей по кредитам чем заемщики, которые не бывали в браке. При этом разведенные и овдовевшие имеют меньше просрочек, чем люди в браке. 
 - Количество детей также связано с вероятностью просрочки по кредиту. Чем больше детей у заемщика тем больше вероятность, что будет просрочка по выплате кредита. Заемщики без детей имеют меньше просрочек по кредитам, чем заемщики с детьми.

[К оглавлению](#content)